# Lesson 5 - Assignment

In this assignment, you will implement a Support Vector Machine Classifier  from scratch and compare the results to existing sklearn algorithm. 

In [1]:
# import packages
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib.legend_handler import HandlerLine2D
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# make this notebook's output stable across runs
np.random.seed(0)

Question 1.1: Implement the cost function cost/objective function:
<img src="https://miro.medium.com/max/688/1*JAS6rUTO7TDlrv4XZSMbsA.png" alt="drawing" width="600"/>


In [2]:
# Why isn't b an input here? Is it an assumed value of 0?
def compute_cost(W, X, Y,reg_strength=1000):
    # TODO calculate cost function
    margin = 1 - Y * (np.dot(X, W))
#     print("margin:", margin)
#     print(type(margin))
#     margin = max(0, margin)
    margin[margin < 0] = 0 
    loss = reg_strength * (np.sum(margin) / X.shape[0])
    cost = 0.5 * np.dot(W, W) + loss
    print("cost:", cost)
    return cost

In [3]:
# margin = [-7.92120974e-01, -1.50753080e-01, -4.15360566e+00, 4.00352650e+00, -1.59051391e+00, -7.35799213e+00, -7.00967938e+00, -6.68023754e+00, 3.57708205e+00, 1.01718300e+00, -4.48967397e-01, -4.85633761e+00]

# margin = np.array(margin)
# margin[margin < 0] = 0 
# margin

Question 1.2: Write a method that calculate the cost gradient:
<img src="https://miro.medium.com/max/866/1*ww3F21VMVGp2NKhm0VTesA.png" alt="drawing" width="600"/>

In [4]:
def calculate_cost_gradient(W, X_batch, Y_batch, reg_strength=1000):
#     print(type(Y_batch))
    if type(Y_batch) == np.int64:
        Y_batch = np.array([Y_batch])
        X_batch = np.array([X_batch])
#     print(type(Y_batch))

    distance = 1 - (Y_batch * np.dot(X_batch, W))
    dw = np.zeros(len(W))

    for ind, d in enumerate(distance):
        if max(0, d) == 0:
            di = W
        else:
            di = W - (reg_strength * Y_batch[ind] * X_batch[ind])
        dw += di

    dw = dw/len(Y_batch)  # average
#     print("dw:", dw)
    return dw

In [5]:
# calculate_cost_gradient( np.array([0.01, 0.01, 0.01, 1]), X_train.to_numpy(), 0.)

Question 1.3: Write a method that performs stochastic Gradient descent as follows:
- Caluclate the gradient of cost function i.e. ∇J(w)
- Update the weights in the opposite direction to the gradient: w = w — ∝(∇J(w))
- Repeat until conversion or until 5000 epochs are reached

In [6]:
# Data here is x train as 2D numpy array
# outputs is y train as 1D numpy array
def sgd(data, outputs, learning_rate = 0.0001, max_epochs = 5000):
    weights = np.zeros(data.shape[1])
    nth = 0
    prev_cost = float("inf")
    cost_threshold = 0.01  # in percent
    # stochastic gradient descent
    for epoch in range(1, max_epochs):
        # shuffle to prevent repeating update cycles
        X, Y = shuffle(data, outputs)
        for ind, x in enumerate(X):#TO DO
            ascent = calculate_cost_gradient(weights, x, Y[ind])
            weights = weights - (learning_rate * ascent)
            # convergence check on 2^nth epoch
            if (epoch == 2 ** nth) or (epoch == max_epochs - 1):
                cost = compute_cost(weights, data, outputs)
                print("Epoch is:{} and Cost is: {}".format(epoch, cost))
                # stoppage criterion
                if abs(prev_cost - cost) < cost_threshold * prev_cost:
                    return weights
                prev_cost = cost
                nth += 1
    
    print("weights:", weights)
    return weights

# Dataset

In [7]:
data = pd.read_csv('data_banknote_authentication.csv')

Y = data.iloc[:, -1]  
X = data.iloc[:, 1:4]
X.insert(loc=len(X.columns), column='intercept', value=1)

Y.replace(to_replace=0,value=-1, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=42)

Question 4: Train and evaluate an SVC using the banknote_authentication data

In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371 entries, 0 to 1370
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   8.6661     1371 non-null   float64
 1   -2.8073    1371 non-null   float64
 2   -0.44699   1371 non-null   float64
 3   intercept  1371 non-null   int64  
dtypes: float64(3), int64(1)
memory usage: 43.0 KB


In [9]:
X.describe() # scaling might help

,8.6661,-2.8073,-0.44699,intercept
count,1371.000000,1371.000000,1371.000000,1371.0
mean,1.917434,1.400694,-1.192200,1.0
std,5.868359,4.310105,2.101683,0.0
min,-13.773100,-5.286100,-8.548200,1.0
25%,-1.711300,-1.553350,-2.417000,1.0
50%,2.313400,0.616630,-0.586650,1.0
75%,6.813100,3.181600,0.394810,1.0
max,12.951600,17.927400,2.449500,1.0


In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [11]:
# train the model
print("training started...")
W = sgd(X_train.to_numpy(), y_train.to_numpy())
# W = sgd(X_train, y_train.to_numpy())
print("training finished.")
print("weights are: {}".format(W))

# testing the model on test set
y_test_predicted = np.array([])
for i in range(X_test.shape[0]):
    yp = np.sign(np.dot(W, X_test.to_numpy()[i]))
#     yp = np.sign(np.dot(W, X_test[i]))
    y_test_predicted = np.append(y_test_predicted, yp)
print("accuracy on test dataset: {}".format(accuracy_score(y_test.to_numpy(), y_test_predicted)))

training started...
cost: 1353.5403433279191
Epoch is:1 and Cost is: 1353.5403433279191
cost: 1361.2123100074366
Epoch is:2 and Cost is: 1361.2123100074366
training finished.
weights are: [-2.29487291 -0.85304504 -1.51191052  1.38837604]
accuracy on test dataset: 0.7522768670309654


[Bonus] Question 5: Train and evaluate an SKLEARN SVC model, and compare the results to your model 

In [12]:
from sklearn.svm import SVC
svc = SVC(gamma='auto')
svc.fit(X_train, y_train)
pred = svc.predict(X_test)
accuracy_score(y_test, pred)

0.9562841530054644

Question 6: Create a new text cell in your Notebook: Complete a 50-100 word summary (or short description of your thinking in applying this week's learning to the solution) of your experience in this assignment. Include: What was your incoming experience with this model, if any? what steps you took, what obstacles you encountered. how you link this exercise to real-world, machine learning problem-solving. (What steps were missing? What else do you need to learn?) This summary allows your instructor to know how you are doing and allot points for your effort in thinking and planning, and making connections to real-world work.

Incoming experience: No incoming experience apart from previous assignments.

Steps taken: This week's lesson was about SVM's. Implementing it helped get a better understanding of what lies under the hood w.r.t. implementation and what kind of data the functions work with.

Obstacles: Got a few type errors and value errors, took a while to fix the errors.

Link to real world: Helped me understand when SVM's can be used versus other algorithms and how SVM's can be scaled to non-linear and multi dimensional problems.

Steps missing (with just this week's learning): I thought data needed to be scaled but accuracy reduced after scaling.